In [ ]:
import numpy as np
import matplotlib.pyplot as plt

m_0 = np.array([5 * 10**3, -20, 5, 20])
p_0 = np.diag([(2 * 10**3) ** 2, 20**2, (2 * 10**3) ** 2, 20**2])
DELTA_T = 1
N_VAL = 200
cov = np.diag([2**2, 2**2])

In [ ]:
def simu_etat(
    delta_t: float, n_val: int, m_0: np.ndarray, sigma: np.ndarray, cov: np.ndarray
):
    x_vec = np.empty(shape=(n_val + 1, 4, 1))

    x_vec[0] = np.random.multivariate_normal(mean=m_0, cov=sigma, size=1).T

    epsilon: np.ndarray = np.random.multivariate_normal(
        mean=[0, 0], cov=cov, size=n_val
    ).reshape((n_val, 2, 1))

    trans_mat = np.array(
        [[1, delta_t, 0, 0], [0, 1, 0, 0], [0, 0, 1, delta_t], [0, 0, 0, 1]]
    )

    cov_mat = np.array(
        [[delta_t**2 / 2, 0], [delta_t, 0], [0, delta_t**2 / 2], [0, delta_t]]
    )

    for k in range(n_val):
        x_vec[k + 1] = trans_mat @ x_vec[k] + cov_mat @ epsilon[k]
    return x_vec

In [ ]:
x_val = simu_etat(delta_t=DELTA_T, n_val=N_VAL, m_0=m_0, sigma=p_0, cov=cov)

In [ ]:
h_mat = np.array([[1, 0, 0, 0], [0, 0, 1, 0]])
r_mat = np.diag([10**2, 10**2])


def simu_obs(x_val: np.ndarray, h_mat: np.ndarray, r_mat: np.ndarray):
    n_val = x_val.shape[0]
    y_vec = np.empty(shape=(n_val, h_mat.shape[0], 1))
    v_val = np.random.multivariate_normal(
        mean=np.zeros(shape=(r_mat.shape[0],)), cov=r_mat, size=n_val
    ).reshape((n_val, r_mat.shape[0], 1))
    for k in range(x_val.shape[0]):
        y_vec[k] = h_mat @ x_val[k] + v_val[k]
    return y_vec

In [ ]:
y_val = simu_obs(x_val=x_val, h_mat=h_mat, r_mat=r_mat)

In [ ]:
def kalman_filter(
    y_vec: np.ndarray,
    f_mat: np.ndarray,
    g_mat: np.ndarray,
    m_0: np.ndarray,
    p_0: np.ndarray,
    c_mat: np.ndarray,
    h_mat: np.ndarray,
    r_mat: np.ndarray,
):
    n_val = y_vec.shape[0]
    x_size = f_mat.shape[0]
    x_vec = np.empty(shape=(n_val, x_size, 1))
    x_vec[0] = m_0.reshape((x_size, 1))
    p_mat = np.empty(shape=(n_val, x_size, x_size))
    p_mat[0] = p_0
    k_mat = np.empty(shape=(n_val, x_size, h_mat.shape[0]))
    id_n = np.eye(x_size)
    for k in range(n_val - 1):
        x_pred = f_mat @ x_vec[k]
        p_pred = f_mat @ p_mat[k] @ (f_mat.T) + g_mat @ c_mat @ (g_mat.T)
        k_mat[k + 1] = (
            p_pred @ (h_mat.T) @ np.linalg.inv(h_mat @ p_pred @ (h_mat.T) + r_mat)
        )
        x_vec[k + 1] = x_pred + k_mat[k + 1] @ (y_vec[k + 1] - h_mat @ x_pred)
        p_mat[k + 1] = (id_n - k_mat[k + 1] @ h_mat) @ p_pred
    return x_vec

In [ ]:
trans_mat = np.array(
    [[1, DELTA_T, 0, 0], [0, 1, 0, 0], [0, 0, 1, DELTA_T], [0, 0, 0, 1]]
)
cov_mat = np.array(
    [[DELTA_T**2 / 2, 0], [DELTA_T, 0], [0, DELTA_T**2 / 2], [0, DELTA_T]]
)

x_hat = kalman_filter(
    y_vec=y_val,
    f_mat=trans_mat,
    g_mat=cov_mat,
    m_0=m_0,
    p_0=p_0,
    c_mat=cov,
    h_mat=h_mat,
    r_mat=r_mat,
)

In [ ]:
%matplotlib inline
START = 0
END = 150
plt.figure(figsize=(15, 15))
plt.plot(x_val[START:END, 0, 0], x_val[START:END, 2, 0], label="X")
plt.plot(y_val[START:END, 0, 0], y_val[START:END, 1, 0], label="Y")
plt.plot(x_hat[START + 1 : END, 0, 0], x_hat[START + 1 : END, 2, 0], label="$\\hat{X}$")
plt.legend(loc="upper center", shadow=True, fontsize="x-large")
plt.show()